In [2]:
from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models import TabNetModelConfig
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import first_analysis

AttributeError: module 'torch' has no attribute 'version'

In [ ]:
def test_regression_TabNet(
    regression_data,
    multi_target,
    continuous_cols,
    categorical_cols,
    continuous_feature_transform,
    normalize_continuous_features,
    target_range,
    batch_size,
    epochs
):
    (train, test, target) = regression_data
    data_config = DataConfig(
        target=target + ["MedInc"] if multi_target else target,
        continuous_cols=continuous_cols,
        categorical_cols=categorical_cols,
        continuous_feature_transform=continuous_feature_transform,
        normalize_continuous_features=normalize_continuous_features,
        #num_workers=num_workers
    )
    model_config_params = {"task": "regression", "metrics":["mean_absolute_percentage_error","mean_absolute_error", "r2_score"]}
    if target_range:
        _target_range = []
        for target in data_config.target:
            _target_range.append(
                (
                    float(train[target].min()),
                    float(train[target].max()),
                )
            )
        model_config_params["target_range"] = _target_range
    model_config = TabNetModelConfig(**model_config_params)
    trainer_config = TrainerConfig(
        max_epochs=epochs,
        checkpoints=None,
        early_stopping=None,
        accelerator="cpu",
        fast_dev_run=False,
        batch_size= batch_size
    )
    optimizer_config = OptimizerConfig()
 
    tabular_model = TabularModel(
        data_config=data_config,
        model_config=model_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
    )
    tabular_model.fit(train=train)
 
    result = tabular_model.evaluate(test)
    #assert "test_mean_squared_error" in result[0].keys()
    pred_df = tabular_model.predict(test)
    assert pred_df.shape[0] == test.shape[0]
    #print(result)
    return result
    

def input_Tab_tests(df_train, df_test, y_train, y_test):
    ref_df_train = pd.DataFrame(df_train)
    ref_df_test = pd.DataFrame(df_test)
 
    ref_df_train.columns = ref_df_train.columns.astype(str)
    ref_df_test.columns = ref_df_test.columns.astype(str)
 
    ref_df_train['Year'] = y_train['Year']
    ref_df_test['Year'] = y_test['Year']

    lista = list(ref_df_train.columns)
    lista_target_range = list(range(1900, 2024))
 
    target_column = str(ref_df_train.columns[-1])
   
    return (ref_df_train, ref_df_test, lista, lista_target_range, target_column)

def hyperparameter(batch_size, epochs, df_train, df_test):
    ref_df_train, ref_df_test, lista, lista_target_range, target_column = input_Tab_tests(df_train, df_test)
    result = test_regression_TabNet(regression_data=(ref_df_train, ref_df_test, [target_column]), multi_target = None,
        continuous_cols = lista,
        categorical_cols = [],
        continuous_feature_transform = None,
        normalize_continuous_features = False,
        target_range=True,
        batch_size=batch_size,
        epoch=epochs )
    return result

In [ ]:
# # DA PROVARE CON DIVERSI DATASET --> FATTO
# X_train = np.load("./Datasets/Train_Val_Test/Scaled/X_train_scaled.npy")
# X_val = np.load("./Datasets/Train_Val_Test/Scaled/X_val_scaled.npy")
# y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [ ]:
# X_train = np.load("./Datasets/Train_Val_Test/Scaled/no_outliers/X_train_scaled.npy")
# X_val = np.load("./Datasets/Train_Val_Test/Scaled/no_outliers/X_val_scaled.npy")
# y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [ ]:
X_train = np.load("./Datasets/Train_Val_Test/PCA/principals_components_train.npy")
X_val = np.load("./Datasets/Train_Val_Test/PCA/principals_components_val.npy")
y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [ ]:
# FATTO
# X_train = np.load("./Datasets/Train_Val_Test/PCA/no_outliers/principals_components_train.npy")
# X_val = np.load("./Datasets/Train_Val_Test/PCA/no_outliers/principals_components_val.npy")
# y_train, y_val = first_analysis.load_df("./Datasets/Train_Val_Test/y_train.csv", "./Datasets/Train_Val_Test/y_val.csv")

In [ ]:
ref_df_train, ref_df_test, lista, lista_target_range, target_column = input_Tab_tests(X_train, X_val, y_train, y_val)

In [ ]:
epochs = list(range(5, 11))
batch_size = list(range(50, 450, 50))
results = []
for j,batch in enumerate(batch_size):
    for i,epoch in enumerate(epochs):
        result = test_regression_TabNet(regression_data=(ref_df_train, ref_df_test, [target_column]), multi_target = None,
            continuous_cols = lista,
            categorical_cols = [],
            continuous_feature_transform = None,
            normalize_continuous_features = False,
            target_range=True,
            batch_size=batch,
            epochs=epoch )
        result[0]["batch_size"] = batch
        result[0]["epochs"] = epoch
        results.append(result)
        print("batch_size: %d / %d      epoch: %d / %d"%(j,(len(batch_size)-1),i,(len(epochs)-1)))

2024-08-06 08:39:36,247 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-08-06 08:39:36,489 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-08-06 08:39:36,679 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-08-06 08:39:37,189 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: TabNetModel

...

2024-08-06 09:30:10,157 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-08-06 09:30:10,161 - {pytorch_tabular.tabular_model:1499} - WARNING - No best model available to load.        
Checkpoint Callback needs to be enabled for this to work

Output()

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


AssertionError: 

In [ ]:
list_dict = [item[0] for item in results]

df_res =pd.DataFrame(list_dict)
df_res.to_csv("./df_Res_tabnet_PCA.csv")

df_res.sort_values(["epochs","batch_size"])